In [14]:
from lib.db_io import Engine

import pandas as pd
import numpy as np
from lib.utils import 구분데이터붙이기, 군집나누기
from lib.simulator_03 import Simulator_01
from scipy.stats import ks_2samp
import warnings
warnings.filterwarnings(action='ignore')

In [23]:
db_info = {'host': '163.152.172.163',
          'port': '5432',
          'db_name': 'postgres',
          'user_name': 'tgsociety',
          'password': 'tgsociety'}

st_dt = '2022-10-01 07:00:00+09'
ed_dt = '2024-08-21 23:00:00+09'
시군구_ids = ['11220', '11230', '11240']

engine = Engine(db_info)


wheres = []
wheres.append('"예약희망일시" >= \'{st_dt}\''.format(st_dt = st_dt))
wheres.append('"예약희망일시" <= \'{ed_dt}\''.format(ed_dt = ed_dt))
wheres.append('"시군구_id" in (\'{}\', \'{}\', \'{}\')'.format(시군구_ids[0], 시군구_ids[1], 시군구_ids[2]))
실제데이터 = engine.select(table = '고객수요_집계구_뷰', schema='datamart', columns = ['시군구_id', '고객_접속일시', '예약희망일시', '예약희망_point']
                     , wheres = wheres
                     , orderbys = ['"예약희망일시" asc'], 
                                   gis_col = '예약희망_point' )
실제데이터

,시군구_id,고객_접속일시,예약희망일시,예약희망_point
0,11220,2022-09-29 17:21:04.756797+00:00,2022-10-01 00:00:00+00:00,POINT (127.01607 37.5056)
1,11230,2022-09-29 13:32:21.396273+00:00,2022-10-01 00:00:00+00:00,POINT (127.04104 37.48994)
2,11230,2022-09-29 21:06:50.370924+00:00,2022-10-01 00:00:00+00:00,POINT (127.03052 37.49679)
3,11240,2022-09-29 18:27:10.246466+00:00,2022-10-01 01:30:00+00:00,POINT (127.12964 37.4901)
4,11240,2022-09-28 02:06:23.213511+00:00,2022-10-01 02:00:00+00:00,POINT (127.10651 37.50329)
...,...,...,...,...
46218,11230,2024-08-20 14:45:57.564497+00:00,2024-08-21 12:00:00+00:00,POINT (127.04923 37.47745)
46219,11220,2024-08-20 14:55:22.487547+00:00,2024-08-21 12:00:00+00:00,POINT (127.00193 37.50224)
46220,11240,2024-08-20 14:08:18.708224+00:00,2024-08-21 12:00:00+00:00,POINT (127.11558 37.51045)
46221,11230,2024-08-20 14:59:55.603373+00:00,2024-08-21 12:00:00+00:00,POINT (127.09295 37.48097)


In [16]:
cur_datas = []
for try_num in range(3):
    simulator = Simulator_01(
                                 simulation_start_dt = st_dt, 
                                 simulation_end_dt = ed_dt,
        data_start_date = st_dt,
                                 data_end_date = ed_dt,
                                 db_info = db_info,
                                 is_first = True,
                                시군구_ids = ['11220', '11230', '11240'], 차량숫자 = 15,
    try_num = try_num)
    cur_datas.append(구분데이터붙이기(simulator.가예약정보.copy(), '예약희망일시'))

In [17]:
cur_datas[0]

,고객_접속일시,예약희망일시,예약희망_point,월,요일,공휴일,시간대,접속연월일,연월일,연
0,2022-09-29 18:24:05.479047+09:00,2022-10-01 09:00:00+09:00,None,10,5,False,조조,20220929,20221001,2022
1,2022-09-30 11:36:09.163286+09:00,2022-10-01 10:00:00+09:00,None,10,5,False,점심,20220930,20221001,2022
2,2022-09-30 12:34:41.917549+09:00,2022-10-01 10:30:00+09:00,None,10,5,False,점심,20220930,20221001,2022
3,2022-09-30 06:04:02.401854+09:00,2022-10-01 11:00:00+09:00,None,10,5,False,점심,20220930,20221001,2022
4,2022-09-29 17:26:40.979965+09:00,2022-10-01 11:00:00+09:00,None,10,5,False,점심,20220929,20221001,2022
...,...,...,...,...,...,...,...,...,...,...
46751,2024-08-20 03:06:56.768483+09:00,2024-08-21 20:30:00+09:00,None,08,2,False,저녁,20240820,20240821,2024
46752,2024-08-20 16:29:53.586032+09:00,2024-08-21 20:30:00+09:00,None,08,2,False,저녁,20240820,20240821,2024
46753,2024-08-12 05:37:25.566320+09:00,2024-08-21 20:30:00+09:00,None,08,2,False,저녁,20240812,20240821,2024
46754,2024-08-20 12:21:36.430190+09:00,2024-08-21 21:00:00+09:00,None,08,2,False,저녁,20240820,20240821,2024


In [18]:
cur_datas[1]

,고객_접속일시,예약희망일시,예약희망_point,월,요일,공휴일,시간대,접속연월일,연월일,연
0,2022-09-30 05:58:23.936549+09:00,2022-10-01 09:00:00+09:00,None,10,5,False,조조,20220930,20221001,2022
1,2022-09-27 14:53:31.208775+09:00,2022-10-01 10:30:00+09:00,None,10,5,False,점심,20220927,20221001,2022
2,2022-09-29 17:42:15.045599+09:00,2022-10-01 10:30:00+09:00,None,10,5,False,점심,20220929,20221001,2022
3,2022-09-30 05:24:35.994709+09:00,2022-10-01 11:00:00+09:00,None,10,5,False,점심,20220930,20221001,2022
4,2022-09-30 05:46:50.547661+09:00,2022-10-01 11:30:00+09:00,None,10,5,False,점심,20220930,20221001,2022
...,...,...,...,...,...,...,...,...,...,...
46169,2024-08-20 04:51:31.691453+09:00,2024-08-21 21:00:00+09:00,None,08,2,False,저녁,20240820,20240821,2024
46170,2024-08-19 02:25:44.537397+09:00,2024-08-21 21:00:00+09:00,None,08,2,False,저녁,20240819,20240821,2024
46171,2024-08-18 20:36:24.716533+09:00,2024-08-21 21:00:00+09:00,None,08,2,False,저녁,20240818,20240821,2024
46172,2024-08-20 17:26:47.039263+09:00,2024-08-21 21:00:00+09:00,None,08,2,False,저녁,20240820,20240821,2024


In [19]:
cur_datas[2]

,고객_접속일시,예약희망일시,예약희망_point,월,요일,공휴일,시간대,접속연월일,연월일,연
0,2022-09-29 19:28:47.889535+09:00,2022-10-01 09:00:00+09:00,None,10,5,False,조조,20220929,20221001,2022
1,2022-09-30 04:25:32.505426+09:00,2022-10-01 09:00:00+09:00,None,10,5,False,조조,20220930,20221001,2022
2,2022-09-29 22:20:54.789744+09:00,2022-10-01 10:30:00+09:00,None,10,5,False,점심,20220929,20221001,2022
3,2022-09-30 01:21:56.783199+09:00,2022-10-01 11:00:00+09:00,None,10,5,False,점심,20220930,20221001,2022
4,2022-09-30 06:51:00.134894+09:00,2022-10-01 11:30:00+09:00,None,10,5,False,점심,20220930,20221001,2022
...,...,...,...,...,...,...,...,...,...,...
46798,2024-08-19 08:36:18.894431+09:00,2024-08-21 21:00:00+09:00,None,08,2,False,저녁,20240819,20240821,2024
46799,2024-08-20 11:48:18.898607+09:00,2024-08-21 21:00:00+09:00,None,08,2,False,저녁,20240820,20240821,2024
46800,2024-08-20 22:05:53.728909+09:00,2024-08-21 21:00:00+09:00,None,08,2,False,저녁,20240820,20240821,2024
46801,2024-08-20 17:18:38.560415+09:00,2024-08-21 21:00:00+09:00,None,08,2,False,저녁,20240820,20240821,2024


In [24]:
ref_data = 구분데이터붙이기(실제데이터.copy(), '예약희망일시')
ref_data['date'] = ref_data['예약희망일시'].dt.date
for i in range(3):
    cur_data = cur_datas[i].copy()
    cur_data['date'] = cur_data['예약희망일시'].dt.date
    ref_counts_list = ref_data['date'].value_counts().sort_index().tolist()
    cur_counts_list = cur_data['date'].value_counts().sort_index().tolist()
    print(ks_2samp(ref_counts_list, cur_counts_list)[1].item())

0.8506336217127587
0.999317102095619
0.4244944369979299


In [5]:

# # execution_date = '2024-07-31 12:00:00+09'

# for try_num in range(18, 100):
#     simulator = Simulator_01(
#                                  simulation_start_dt = st_dt, 
#                                  simulation_end_dt = ed_dt,
#         data_start_date = st_dt,
#                                  data_end_date = ed_dt,
#                                  db_info = db_info,
#                                  is_first = True,
#                                 시군구_ids = ['11220', '11230', '11240'], 차량숫자 = 15,
#     try_num = try_num)
#     # init_states = simulator.make_init_states(sample_num = 1000)
#     ref_data = 구분데이터붙이기(simulator.data.copy(), '예약희망일시')
#     cur_data = 구분데이터붙이기(simulator.가예약정보.copy(), '예약희망일시')
    
#     ref_data['date'] = ref_data['예약희망일시'].dt.date
#     cur_data['date'] = cur_data['예약희망일시'].dt.date
    
#     군집_ref_data = 군집나누기(ref_data.copy(), simulator.cluster.군집s)
#     군집_cur_data = 군집나누기(cur_data.copy(), simulator.cluster.군집s)
    
#     p_values = []
#     for key in list(군집_ref_data):
#         if (len(군집_ref_data[key])==0) & (len(군집_cur_data[key])==0):
#             p_values.append(1)
#         elif (len(군집_ref_data[key])==0) | (len(군집_cur_data[key])==0):
#             p_values.append(0)
#         else:
#             p_values.append(ks_2samp(군집_ref_data[key], 군집_cur_data[key])[1].item())
    
#     분자 = []
#     분모 = []
#     keys = list(군집_ref_data)
#     for i in range(len(p_values)):
#         분자.append(p_values[i]*len(군집_ref_data[keys[i]]))
#         분모.append(len(군집_ref_data[keys[i]]))
#     print(try_num, sum(분자)/sum(분모))

#     ref_counts_list = ref_data['date'].value_counts().sort_index().tolist()
#     cur_counts_list = cur_data['date'].value_counts().sort_index().tolist()
#     print(ks_2samp(ref_counts_list, cur_counts_list)[1].item())

성능지표 2. 시뮬레이션 성능의 측정 목적은 시뮬레이션에서 생성된 데이터가 실제 데이터와 얼마나 유사한지를 평가하는 것입니다. 이를 위해 Kolmogorov-Smirnov 2-sample 테스트를 수행하여, 귀무가설인 '두 표본 데이터 세트가 동일한 분포를 따른다'를 통계적으로 유의미하게 기각할 수 있는지를 확인하기 위해 p-value가 0.05 이하인지 관찰합니다. 만약 p-value가 0.05 이하라면 귀무가설이 기각되며, 이는 시뮬레이션에서 생성된 데이터가 실제 데이터와 유의미하게 다르다는 것을 의미합니다. 따라서 p-value가 0.05 이하로 나타나지 않도록 하는 것이 본 과제의 목표였습니다.

그러나, 과제 계획서 작성 시 부주의로 인해 목표치가 'p-value 0.05 이하'로 설정되었습니다. 이에 따라 실험 의도에 부합하도록 목표치를 'p-value 0.05 이상'으로 재설정해야 합니다. p-value는 두 표본이 동일한 모분포에서 추출되었다고 가정할 때, 관찰된 KS 검정 통계량보다 더 극단적인 값이 나타날 확률을 의미합니다.

또한, 기준값을 통상적으로 사용하는 통계적으로 유의함을 의미하는 0.05에서, 조금 더 보수적으로 상향조정을 하고자 합니다.
시뮬레이터 성능을 평가하기 위해 시뮬레이션 모델에서 생성된 데이터와 실제 데이터 간의 분포 차이를 KS 테스트로 측정한 연구로는, p-value 0.05를 중요 수준으로 설정한 연구로 'Yan, Xuedong, et al. (2008)'과 'Toledo, Tomer, et al. (2004)'이 있으며, p-value 0.2를 기준으로 사용한 연구로는 'Rao, Lei, et al. (1998)'이 있습니다.

본 과제에서는 조금 더 보수적인 접근을 위해 'Rao, Lei, et al. (1998)' 연구처럼 기준치를 p-value 0.2로 상향 조정하였습니다. 따라서, 성능지표 2에 대한 연구개발 목표치를 기존의 'p-value 0.05 이하'에서 'p-value 0.2 이상'으로 수정하고자 합니다.

참조:
Yan, Xuedong, et al. "Validating a driving simulator using surrogate safety measures." Accident Analysis & Prevention 40.1 (2008): 274-288.  
Toledo, Tomer, and Haris N. Koutsopoulos. "Statistical validation of traffic simulation models." Transportation Research Record 1876.1 (2004): 142-150.  
Rao, Lei, Larry Owen, and David Goldsman. "Development and application of a validation framework for traffic simulation models." 1998 Winter Simulation Conference. Proceedings (Cat. No. 98CH36274). Vol. 2. IEEE, 1998.

Yan, Xuedong, et al. "Validating a driving simulator using surrogate safety measures." Accident Analysis & Prevention 40.1 (2008): 274-288.  
0.05 기준  
Toledo, Tomer, and Haris N. Koutsopoulos. "Statistical validation of traffic simulation models." Transportation Research Record 1876.1 (2004): 142-150.  
Rao, Lei, Larry Owen, and David Goldsman. "Development and application of a validation framework for traffic simulation models." 1998 Winter Simulation Conference. Proceedings (Cat. No. 98CH36274). Vol. 2. IEEE, 1998.  

In [6]:
ks_2samp(ref_counts_list, cur_counts_list)[1].item()

1.779808786987362e-64

In [7]:
p_values

NameError: name 'p_values' is not defined

In [ ]:
len(list(군집_cur_data))

In [ ]:
군집_ref_data

In [ ]:
17 클러스터별
